In [1]:
%matplotlib qt5

import matplotlib.pyplot as plt
import numpy as np
import mne
import os
import random

from mne import io, preprocessing
from mne.stats import permutation_cluster_test

In [7]:
data_path = os.path.join(os.getcwd(), 'processed_epochs')
tmin = -0.2 # start of each epoch (400ms before the trigger)
tmax = 1 # end of each epoch (1000ms after the trigger)
threshold = 4

mne.set_log_level('ERROR')

In [8]:
files = []
for f in os.listdir(data_path):
    if f.endswith('.fif'):
        files.append(f)
print(files)

['990291-epo.fif', '382733-epo.fif', '477819-epo.fif', '554432_2-epo.fif', '477819_2-epo.fif', '382733_2-epo.fif', '554432-epo.fif', '884723-epo.fif', '884723_2-epo.fif', '990291_2-epo.fif', '928376_2-epo.fif', '928376-epo.fif']


In [12]:
count = 0
results = []
for file in files:
    epochs = mne.read_epochs(data_path + f'/{file}')
    events = [*epochs.event_id.keys()]
    for i in range(len(events)-1):
        for j in range(i+1, len(events)):
            first = events[i]
            epochs1 = epochs[first]
            condition1 = epochs1.get_data()  # as 3D matrix
            
            second = events[j]
            epochs2 = epochs[second]
            condition2 = epochs2.get_data()  # as 3D matrix

            condition1 = condition1[:, 0, :]  # take only one channel to get a 2D array
            condition2 = condition2[:, 0, :]  # take only one channel to get a 2D array

            T_obs, clusters, cluster_p_values, H0 = \
                permutation_cluster_test([condition1, condition2], n_permutations=1000,
                                         threshold=threshold, tail=1, n_jobs=1)            
            count += 1

            for i_c, c in enumerate(clusters):
                c = c[0]
                if cluster_p_values[i_c] < 0.05:
                    results.append(f'{file}: condition1 = {first}, condition2 = {second}, p-value: {cluster_p_values[i_c]}')
print(f'total: {count}')
                    

<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
  threshold=threshold, tail=1, n_jobs=1)
<ipython-input-12-948a6fbc4739>:21:

total: 375


In [13]:
for r in results:
    print(r)

382733-epo.fif: condition1 = Finnish Conf, condition2 = Favourite Match, p-value: 0.09
382733-epo.fif: condition1 = Neutrals, condition2 = Favourite Match, p-value: 0.089
382733-epo.fif: condition1 = Neutrals, condition2 = Favourite Match, p-value: 0.089
382733-epo.fif: condition1 = Neutrals, condition2 = Favourite Match, p-value: 0.089
477819-epo.fif: condition1 = Foreing Match, condition2 = Foreing Conf, p-value: 0.032
554432_2-epo.fif: condition1 = Finnish Match, condition2 = Dislike Conf, p-value: 0.08
554432_2-epo.fif: condition1 = Foreing Conf, condition2 = Neutrals, p-value: 0.078
477819_2-epo.fif: condition1 = Finnish Match, condition2 = Finnish Conf, p-value: 0.002
477819_2-epo.fif: condition1 = Finnish Match, condition2 = Foreing Conf, p-value: 0.092
477819_2-epo.fif: condition1 = Finnish Match, condition2 = Favourite Match, p-value: 0.044
477819_2-epo.fif: condition1 = Finnish Match, condition2 = Unknowns, p-value: 0.04
477819_2-epo.fif: condition1 = Finnish Conf, condition2

In [27]:

epochs = mne.read_epochs(data_path + '/477819-epo.fif')
first = 'Foreing Match'
second = 'Foreing Conf'

epochs1 = epochs[first]
condition1 = epochs1.get_data()  # as 3D matrix

# matching
epochs2 = epochs[second]
condition2 = epochs2.get_data()  # as 3D matrix

condition1 = condition1[:, 0, :]  # take only one channel to get a 2D array
condition2 = condition2[:, 0, :]  # take only one channel to get a 2D array

T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([condition1, condition2], n_permutations=1000,
                             threshold=threshold, tail=1, n_jobs=1)       

In [28]:
times = epochs1.times
plt.close('all')
plt.subplot(211)
plt.title(f'{first} vs. {second}')
plt.plot(times, condition1.mean(axis=0) - condition2.mean(axis=0),
         label=f'ERF Contrast ({first} - {second})')
plt.ylabel("MEG (T / m)")
plt.legend()
plt.subplot(212)
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] <= 0.1:
        h = plt.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
#     else:
#         plt.axvspan(times[c.start], times[c.stop - 1], color=(0.3, 0.3, 0.3),
#                     alpha=0.3)
hf = plt.plot(times, T_obs, 'g')
plt.legend((h, ), ('cluster p-value < 0.05', ))
plt.xlabel("time (ms)")
plt.ylabel("f-values")
plt.show()